In [1]:
import os 
from datasets import load_dataset
import json

data = "/Users/marisa/clausal-coordinate-ellipsis/german-common-crawl/de_de_pairs.jsonl"

de_de_dataset = load_dataset("json", data_files=data, split='train')

In [2]:
from responses import target
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

source_lang = "de"
target_lang = "de"
prefix = "translate German to German: "

In [3]:
de_de_dataset = de_de_dataset.filter(lambda example: len(example['text']) >= 20)
de_de_dataset = de_de_dataset.filter(lambda example: len(example['gold_sentence']) >= 20)

In [4]:
def preprocess_function(examples):
    inputs = prefix + examples['text']
    targets = examples["gold_sentence"]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True, padding='longest', return_tensors='pt')
    return model_inputs

In [5]:
tokenized_dataset = de_de_dataset.map(preprocess_function, batched=False)

In [6]:
def correct_inputs_masks_labels(examples):
    examples['input_ids'] = examples['input_ids'][0] 
    examples['attention_mask'] = examples['attention_mask'][0] 
    examples['labels'] = examples['labels'][0] 
    return examples

In [7]:
tokenized_dataset = tokenized_dataset.map(correct_inputs_masks_labels, batched=False)

In [8]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2)

In [9]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [10]:
import evaluate
import numpy as np 

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [11]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [12]:
training_args = Seq2SeqTrainingArguments(
    output_dir="de_de_040124",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=False, # set true when cuda available
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [13]:
trainer.train()

  0%|          | 0/93076 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.0236, 'learning_rate': 1.989256091795952e-05, 'epoch': 0.01}
{'loss': 0.0117, 'learning_rate': 1.9785121835919034e-05, 'epoch': 0.02}
{'loss': 0.0086, 'learning_rate': 1.9677682753878555e-05, 'epoch': 0.03}
{'loss': 0.0082, 'learning_rate': 1.957024367183807e-05, 'epoch': 0.04}
{'loss': 0.0077, 'learning_rate': 1.9462804589797586e-05, 'epoch': 0.05}
{'loss': 0.0069, 'learning_rate': 1.93553655077571e-05, 'epoch': 0.06}
{'loss': 0.007, 'learning_rate': 1.924792642571662e-05, 'epoch': 0.08}
{'loss': 0.0059, 'learning_rate': 1.9140487343676136e-05, 'epoch': 0.09}
{'loss': 0.0067, 'learning_rate': 1.9033048261635654e-05, 'epoch': 0.1}
{'loss': 0.0051, 'learning_rate': 1.892560917959517e-05, 'epoch': 0.11}
{'loss': 0.0052, 'learning_rate': 1.881817009755469e-05, 'epoch': 0.12}
{'loss': 0.0054, 'learning_rate': 1.8710731015514206e-05, 'epoch': 0.13}
{'loss': 0.0055, 'learning_rate': 1.860329193347372e-05, 'epoch': 0.14}
{'loss': 0.0056, 'learning_rate': 1.8495852851433238e-05, 'ep

RuntimeError: MPS backend out of memory (MPS allocated: 3.36 GB, other allocations: 14.68 GB, max allowed: 18.13 GB). Tried to allocate 251.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).